In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

In [18]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from itertools import combinations
import random

def compute_error_rates(true_labels, pred_labels):
    """Compute APCER, BPCER, ACER."""
    true_labels = np.array(true_labels)
    pred_labels = np.array(pred_labels)

    # Define mapping
    labels = [0, 1]  # Ensure consistent order

    # Compute confusion matrix
    cm = confusion_matrix(true_labels, pred_labels, labels=labels)
    #print(cm)
    TN, FP, FN, TP = cm.ravel()  # Unpack the matrix

    apcer = FN/(FN+TP)
    bpcer = FP/(FP+TN)
    acer = 0.5 * (apcer + bpcer)
    return acer, apcer, bpcer

def get_fold_data(df, train_folds: list = [], val_folds: list = [], df_syn = pd.DataFrame(), amount_syn = 0):
    amount_auth = max(0, len(train_folds) - amount_syn)
    auth_folds = train_folds[:amount_auth]
    syn_folds = train_folds[amount_auth:]
    
    train_df_auth = df[df['fold'].isin(auth_folds)]
    if not df_syn.empty:
        train_df_syn = df_syn[df_syn['fold'].isin(syn_folds)]
        train_df = pd.concat([train_df_auth, train_df_syn])
    else:
        train_df = train_df_auth
    
    val_df = df[df['fold'].isin(val_folds)]

    X_train = np.stack(train_df['features'].values)
    X_val = np.stack(val_df['features'].values)

    y_train = train_df['label']
    y_val = val_df['label']

    return X_train, y_train, X_val, y_val

def to_result_tuple(k, method, category, acer, apcer, bpcer):
    return [
        (k, method, 'ACER', category, acer),
        (k, method, 'APCER', category, apcer),
        (k, method, 'BPCER', category, bpcer),
    ]

def crossvalidation(df, k=5, method='xxxx', df_syn = pd.DataFrame()):
    folds = {0, 1, 2, 3, 4}
    errors = []
    
    for i in range(0, 5):
        train_folds_list = list({0, 1, 2, 3, 4} - {i})
        random.shuffle(train_folds_list)

        # train = 4/5, val = 1/5
        X_train, y_train, X_val, y_val = get_fold_data(df, train_folds = train_folds_list[:4], val_folds=[i])
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        predictions = knn.predict(X_val)
        errors.extend(to_result_tuple(k, method, '4/5', *compute_error_rates(y_val, predictions)))

        # train = 2/5, val = 1/5
        X_train, y_train, X_val, y_val = get_fold_data(df, train_folds = train_folds_list[:2], val_folds=[i])
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        predictions = knn.predict(X_val)
        errors.extend(to_result_tuple(k, method, '2/5', *compute_error_rates(y_val, predictions)))

        # train = 1/5, val = 1/5
        X_train, y_train, X_val, y_val = get_fold_data(df, train_folds = train_folds_list[:1], val_folds=[i])
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_val, y_val)
        predictions = knn.predict(X_train)
        errors.extend(to_result_tuple(k, method, '1/5', *compute_error_rates(y_val, predictions)))

        if not df_syn.empty:
            # train = 1+1/5, val = 1/5
            X_train, y_train, X_val, y_val = get_fold_data(df, train_folds = train_folds_list[:2], val_folds=[i], df_syn = df_syn, amount_syn = 1)
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_val, y_val)
            predictions = knn.predict(X_train)
            errors.extend(to_result_tuple(k, method, '(1+1)/5', *compute_error_rates(y_val, predictions)))

            # train = 1+3/5, val = 1/5
            X_train, y_train, X_val, y_val = get_fold_data(df, train_folds = train_folds_list[:4], val_folds=[i], df_syn = df_syn, amount_syn = 3)
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(X_val, y_val)
            predictions = knn.predict(X_train)
            errors.extend(to_result_tuple(k, method, '(1+3)/5', *compute_error_rates(y_val, predictions)))

    return errors

In [4]:
dataset_title = 'IDIAP'
metrics = ['ACER', 'APCER', 'BPCER']
ks = [3, 5, 7, 9]
lbp_settings = [
    { 'P': 8, 'R': 1, 'method': 'default', 'title': 'Default' },
    { 'P': 8, 'R': 1, 'method': 'ror', 'title': 'Rotation Invariant' },
    { 'P': 8, 'R': 1, 'method': 'uniform', 'title': 'Uniform' },
    { 'P': 8, 'R': 1, 'method': 'nri_uniform', 'title': 'Uniform Grayscale Invariant' },
    { 'P': 8, 'R': 1, 'method': 'var', 'title': 'Contrast Variance' },
]

In [8]:
feature_dfs = [pd.read_csv(file) for file in [f'{dataset_title}_lbp-{s['method']}.csv' for s in lbp_settings]]
feature_dfs_vae = [pd.read_csv(file) for file in [f'{dataset_title}-VAE_lbp-{s['method']}.csv' for s in lbp_settings]]
feature_dfs_ae = [pd.read_csv(file) for file in [f'{dataset_title}-AE_lbp-{s['method']}.csv' for s in lbp_settings]]

In [31]:
feature_dfs[0]['features

array(['[1.10063158e-01 1.84240602e-02 4.57142857e-03 6.52631579e-03\n 2.03368421e-02 2.61052632e-03 9.12481203e-03 5.46165414e-03\n 1.82255639e-03 8.54135338e-04 4.39097744e-04 1.15488722e-03\n 4.41503759e-03 1.03458647e-03 9.19699248e-03 1.10436090e-02\n 1.63368421e-02 2.88721805e-03 1.81052632e-03 2.11127820e-03\n 6.00902256e-03 4.75187970e-04 2.73684211e-03 1.28120301e-03\n 2.57443609e-03 8.54135338e-04 8.48120301e-04 2.07518797e-03\n 5.45563910e-03 9.44360902e-04 1.13203008e-02 1.65172932e-02\n 4.61353383e-03 1.79248120e-03 2.52631579e-04 6.43609023e-04\n 4.09022556e-04 1.08270677e-04 2.22556391e-04 2.28571429e-04\n 3.36842105e-04 3.06766917e-04 9.02255639e-05 3.12781955e-04\n 3.12781955e-04 1.32330827e-04 1.80451128e-04 5.29323308e-04\n 7.04962406e-03 1.75037594e-03 5.47368421e-04 9.68421053e-04\n 1.29924812e-03 1.26315789e-04 6.49624060e-04 3.12781955e-04\n 3.54887218e-03 1.85864662e-03 9.02255639e-04 2.24360902e-03\n 2.95338346e-03 6.91729323e-04 3.78345865e-03 5.99699248e-03\n

In [19]:
def plot(dfs_auth, dfs_syn):
    data = []
    for auth, syn, setting in zip(dfs_auth, dfs_syn, lbp_settings):
        for k in ks:
            data.extend(crossvalidation(auth, k, setting['method'], syn))

    return pd.DataFrame(data)

In [20]:
plot(feature_dfs, feature_dfs_vae)

ValueError: Expected 2D array, got 1D array instead:
array=['[1.10063158e-01 1.84240602e-02 4.57142857e-03 6.52631579e-03\n 2.03368421e-02 2.61052632e-03 9.12481203e-03 5.46165414e-03\n 1.82255639e-03 8.54135338e-04 4.39097744e-04 1.15488722e-03\n 4.41503759e-03 1.03458647e-03 9.19699248e-03 1.10436090e-02\n 1.63368421e-02 2.88721805e-03 1.81052632e-03 2.11127820e-03\n 6.00902256e-03 4.75187970e-04 2.73684211e-03 1.28120301e-03\n 2.57443609e-03 8.54135338e-04 8.48120301e-04 2.07518797e-03\n 5.45563910e-03 9.44360902e-04 1.13203008e-02 1.65172932e-02\n 4.61353383e-03 1.79248120e-03 2.52631579e-04 6.43609023e-04\n 4.09022556e-04 1.08270677e-04 2.22556391e-04 2.28571429e-04\n 3.36842105e-04 3.06766917e-04 9.02255639e-05 3.12781955e-04\n 3.12781955e-04 1.32330827e-04 1.80451128e-04 5.29323308e-04\n 7.04962406e-03 1.75037594e-03 5.47368421e-04 9.68421053e-04\n 1.29924812e-03 1.26315789e-04 6.49624060e-04 3.12781955e-04\n 3.54887218e-03 1.85864662e-03 9.02255639e-04 2.24360902e-03\n 2.953383

In [ ]:
fig, axes = plt.subplots(nrows=len(ks), ncols=len(feature_dfs), figsize=(12, 12), sharey=True)
fig.suptitle('Error Rates Across Different Number of Neighbours and LBPs (IDIAP)', fontsize=14)

for r, k in enumerate(ks):
    ax = axes[r]
    if r == 0:
        ax.set_title(lbp_settings[c]['title'])
    errors_df = pd.DataFrame(crossvalidation(feature_dfs[0], k=k, method=df_syn=feature_dfs_vae[0]), columns=metrics)
    sns.barplot(data=errors_df, capsize=.1, ax=ax)

plt.tight_layout()
plt.show()

In [12]:
errors = []
errors.extend(to_result_tuple(3, 'ROR', '1/5', 0.5, 0.1, 0.3))
errors

[(3, 'ROR', 'ACER', '1/5', 0.5),
 (3, 'ROR', 'APCER', '1/5', 0.1),
 (3, 'ROR', 'BPCER', '1/5', 0.3)]